# 🎯 Mitsui Commodity Prediction - Production Model

Optimized ML model for predicting 424 commodity targets across multiple global markets (LME, JPX, US, FX).
Uses **XGBoost** regression with advanced feature engineering for rank correlation Sharpe ratio optimization.

In [ ]:
# Essential imports

import warnings
import numpy as np
import polars as pl
import pandas as pd
import os
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr

warnings.filterwarnings('ignore')
NUM_TARGET_COLUMNS = 424

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copying files from /content/drive/MyDrive/Colab Notebooks/mitsui-commodity-prediction-challenge/ to /content/
Copied '/content/drive/MyDrive/Colab Notebooks/mitsui-commodity-prediction-challenge/kaggle_evaluation' to '/content/kaggle_evaluation'
Copied '/content/drive/MyDrive/Colab Notebooks/mitsui-commodity-prediction-challenge/lagged_test_labels' to '/content/lagged_test_labels'
Copied 'train.csv' to '/content/'
Copied 'test.csv' to '/content/'
Copied 'target_pairs.csv' to '/content/'
Copied 'train_labels.csv' to '/content/'
Copied 'submission.parquet' to '/content/'
Copied 'model_optimized.ipynb' to '/content/'


In [ ]:
import kaggle_evaluation.mitsui_inference_server

In [ ]:
def create_features(df):
    """Advanced feature engineering for commodity prediction"""

    # Base features (excluding date_id and targets)
    feature_cols = [col for col in df.columns
                   if not col.startswith('target_') and col != 'date_id']
    X = df[feature_cols].copy()

    # Handle missing values
    X = X.fillna(method='ffill').fillna(X.median())

    # 1. Lagged features (1-5 days)
    original_cols = X.columns.tolist()
    for lag in range(1, 6):
        for col in original_cols[:50]:  # Limit to prevent explosion
            X[f'{col}_lag_{lag}'] = X[col].shift(lag)

    # 2. Rolling statistics (5, 10, 20 day windows)
    windows = [5, 10, 20]
    for window in windows:
        for col in original_cols[:30]:
            X[f'{col}_ma_{window}'] = X[col].rolling(window).mean()
            X[f'{col}_std_{window}'] = X[col].rolling(window).std()

    # 3. Momentum features
    for col in original_cols[:20]:
        X[f'{col}_momentum_5'] = X[col] / X[col].shift(5) - 1
        X[f'{col}_momentum_10'] = X[col] / X[col].shift(10) - 1

    # 4. Cross-market ratios (LME metals)
    lme_cols = [col for col in original_cols if col.startswith('LME_') and col.endswith('_Close')]
    for i, col1 in enumerate(lme_cols[:5]):
        for col2 in lme_cols[i+1:6]:
            X[f'{col1}_vs_{col2}_ratio'] = X[col1] / (X[col2] + 1e-8)

    # 5. Volatility features
    for col in original_cols[:15]:
        X[f'{col}_volatility_10'] = X[col].rolling(10).std() / (X[col].rolling(10).mean() + 1e-8)

    # Final cleanup
    X = X.fillna(X.median())
    return X

In [ ]:
def rank_correlation_score(y_true, y_pred):
    """Calculate rank correlation Sharpe ratio (competition metric)"""
    daily_correlations = []

    for i in range(len(y_true)):
        true_row = y_true[i]
        pred_row = y_pred[i]

        # Find non-null values
        mask = ~np.isnan(true_row)
        if mask.sum() < 2:
            continue

        true_vals = true_row[mask]
        pred_vals = pred_row[mask]

        # Calculate rank correlation
        if np.std(true_vals) > 0 and np.std(pred_vals) > 0:
            corr, _ = spearmanr(true_vals, pred_vals)
            if not np.isnan(corr):
                daily_correlations.append(corr)

    if len(daily_correlations) == 0:
        return 0.0

    daily_correlations = np.array(daily_correlations)
    mean_corr = np.mean(daily_correlations)
    std_corr = np.std(daily_correlations)

    return mean_corr / std_corr if std_corr != 0 else 0.0

In [ ]:
# Load and prepare training data
print("Loading training data...")
train_df = pd.read_csv('train.csv')
train_labels_df = pd.read_csv('train_labels.csv')
train_merged = train_df.merge(train_labels_df, on='date_id', how='inner')

print(f"Training data shape: {train_merged.shape}")

# Prepare features and targets
target_cols = [col for col in train_merged.columns if col.startswith('target_')]
print(f"Creating {len(target_cols)} target predictions...")

X_train_raw = create_features(train_merged)
y_train = train_merged[target_cols].values

# Handle missing targets
target_medians = np.nanmedian(y_train, axis=0)
y_train_filled = np.where(np.isnan(y_train), target_medians, y_train)

print(f"Feature engineering complete: {X_train_raw.shape[1]} features")

Loading training data...
Training data shape: (1917, 982)
Creating 424 target predictions...
Feature engineering complete: 1048 features


In [ ]:
# Train-validation split (time series)
val_split = len(X_train_raw) - 200
X_train = X_train_raw.iloc[:val_split]
X_val = X_train_raw.iloc[val_split:]
y_train_split = y_train_filled[:val_split]
y_val_split = y_train_filled[val_split:]

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Clean data
X_train_scaled = np.nan_to_num(X_train_scaled, nan=0.0, posinf=0.0, neginf=0.0)
X_val_scaled = np.nan_to_num(X_val_scaled, nan=0.0, posinf=0.0, neginf=0.0)

print(f"Training samples: {len(X_train_scaled)}")
print(f"Validation samples: {len(X_val_scaled)}")

Training samples: 1717
Validation samples: 200


In [ ]:
# Train XGBoost model (optimized for rank correlation)
print("Training XGBoost model...")

# Configure XGBoost for FAST validation (reduced parameters)
xgb_base = xgb.XGBRegressor(
    n_estimators=50,         # Réduit de 200 à 50 pour validation rapide
    learning_rate=0.1,       # Augmenté de 0.05 à 0.1 pour compenser moins d'arbres
    max_depth=4,             # Réduit de 6 à 4 pour arbres plus simples
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    tree_method='hist',      # Méthode la plus rapide
    reg_alpha=0.1,
    reg_lambda=0.1
)

xgb_model = MultiOutputRegressor(xgb_base)
print("Starting validation training (fast mode)...")
xgb_model.fit(X_train_scaled, y_train_split)
print("Validation training complete!")

xgb_pred = xgb_model.predict(X_val_scaled)

# Evaluate
xgb_score = rank_correlation_score(y_val_split, xgb_pred)
print(f"XGBoost - Rank Correlation Sharpe: {xgb_score:.4f}")

# Select best model
best_model = xgb_model
best_score = xgb_score

print(f"Best model performance: {best_score:.4f}")
print("⚡ Validation completed in fast mode!")

Training XGBoost model...
Starting validation training (fast mode)...


In [ ]:
# Train final model on complete dataset
print("Training final model on complete dataset...")

# Scale all data
final_scaler = StandardScaler()
X_full_scaled = final_scaler.fit_transform(X_train_raw)
X_full_scaled = np.nan_to_num(X_full_scaled, nan=0.0, posinf=0.0, neginf=0.0)

# Train final XGBoost model (FULL POWER parameters)
print("Using optimized parameters for production model...")
final_model = MultiOutputRegressor(
    xgb.XGBRegressor(
        n_estimators=150,        # Bon compromis entre performance et vitesse
        learning_rate=0.08,      # Équilibré
        max_depth=5,             # Bon compromis
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        tree_method='hist',
        reg_alpha=0.1,
        reg_lambda=0.1
    )
)

print("Starting final training...")
final_model.fit(X_full_scaled, y_train_filled)
print("Final model training complete!")
print("🚀 Production model ready with optimized parameters!")

In [ ]:
# Production prediction function
def predict(
    test: pl.DataFrame,
    label_lags_1_batch: pl.DataFrame,
    label_lags_2_batch: pl.DataFrame,
    label_lags_3_batch: pl.DataFrame,
    label_lags_4_batch: pl.DataFrame,
) -> pl.DataFrame:
    """ML-powered prediction function using trained XGBoost model"""

    try:
        # Convert to pandas if needed
        if isinstance(test, pl.DataFrame):
            test_df = test.to_pandas()
        else:
            test_df = test.copy()

        # Apply feature engineering
        X_test = create_features(test_df)

        # Scale features
        X_test_scaled = final_scaler.transform(X_test)
        X_test_scaled = np.nan_to_num(X_test_scaled, nan=0.0, posinf=0.0, neginf=0.0)

        # Make predictions
        predictions_array = final_model.predict(X_test_scaled)

        # Convert to DataFrame
        predictions_dict = {f'target_{i}': predictions_array[0, i] for i in range(NUM_TARGET_COLUMNS)}
        predictions = pl.DataFrame(predictions_dict)

        return predictions

    except Exception as e:
        print(f"Prediction error: {e}")
        # Fallback predictions
        predictions = pl.DataFrame({f'target_{i}': i / 1000 for i in range(NUM_TARGET_COLUMNS)})
        return predictions

# Test prediction function
test_sample = train_df.tail(1)
test_result = predict(
    pl.DataFrame(test_sample),
    pl.DataFrame(), pl.DataFrame(), pl.DataFrame(), pl.DataFrame()
)
print(f"✅ Prediction test successful! Shape: {test_result.shape}")
print(f"Sample predictions: {test_result.to_pandas().iloc[0, :5].values}")

In [ ]:
# Initialize inference server
inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    # Run local gateway for testing
    inference_server.run_local_gateway(('.',))

print("🎉 Mitsui Commodity Prediction Model Ready!")
print(f"📊 Features: {X_full_scaled.shape[1]} | Targets: {NUM_TARGET_COLUMNS}")
print(f"🏆 Validation Score: {best_score:.4f} (Rank Correlation Sharpe)")
print("🚀 Using XGBoost for superior gradient boosting performance!")